In [ ]:
import pandas as pd

file_path = "C:/Users/jihoo/Blitz/order_report__2.xlsx"
df = pd.read_excel(file_path)
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
# 결측치 확인
missing_data = df.isnull().sum()
print(missing_data)


In [ ]:
# 결측치 비율 확인
missing_percentage = (df.isnull().sum() / len(df)) * 100
print(missing_percentage)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Weight 분포
sns.histplot(df['Weight'], bins=20, kde=True)
plt.title('Weight Distribution')
plt.show()

# Service Type 분포
sns.countplot(data=df, x='Service Type')
plt.title('Service Type Distribution')
plt.show()

# Payment Type 분포
sns.countplot(data=df, x='Payment Type')
plt.title('Payment Type Distribution')
plt.show()


In [ ]:
pip install geopy


In [ ]:
# df_selected가 기존의 데이터프레임이라면 이를 정의해 주세요
# 예를 들어 df라는 데이터프레임이 이미 존재할 경우 이를 df_selected로 정의합니다.
df_selected = df.copy()  # 혹은 실제 데이터를 불러오는 코드 사용

# 결측치가 없는 행만 선택하여 df_filtered 생성
df_filtered = df_selected.dropna(subset=['Created At', 'Dropoff done at'])

# 'Created At'과 'Dropoff done at'을 datetime 형식으로 변환
df_filtered['Created At'] = pd.to_datetime(df_filtered['Created At'])
df_filtered['Dropoff done at'] = pd.to_datetime(df_filtered['Dropoff done at'], errors='coerce')

# 배달 시간 계산 (시간 차이)
df_filtered['delivery_time'] = (df_filtered['Dropoff done at'] - df_filtered['Created At']).dt.total_seconds() / 3600

# 배달 시간의 분포 확인
sns.histplot(df_filtered['delivery_time'].dropna(), bins=20)
plt.title('Delivery Time Distribution (Hours)')
plt.xlabel('Delivery Time (Hours)')
plt.ylabel('Frequency')
plt.show()


In [ ]:
df.columns

In [10]:
selected_columns = [
    'AWB Number',              # 주문 번호 (식별자)
    'Weight',                  # 화물 무게
    'Service Type',            # 서비스 유형
    'COD Amount',              # COD 여부 및 금액
    'Sender Latitude',         # 발송자 위치 (위도)
    'Sender Longitude',        # 발송자 위치 (경도)
    'Consignee Latitude',      # 수령자 위치 (위도)
    'Consignee Longitude',     # 수령자 위치 (경도)
    'Created At',              # 주문 생성 시간
    'Dropoff done at',         # 드롭오프 완료 시간
    'Pickup done At (1st Attempt)', # 첫 픽업 시도 시간
    'Status (Internal)',       # 주문 상태
    'Failed Reason',           # 실패 이유 (선택 사항)
]
df_selected = df[selected_columns]


In [ ]:
import pandas as pd

# 시간 변수를 datetime 형식으로 변환
df_selected['Created At'] = pd.to_datetime(df_selected['Created At'])
df_selected['Dropoff done at'] = pd.to_datetime(df_selected['Dropoff done at'], errors='coerce')

# 배달 소요 시간 계산 (드롭오프 완료 시간 - 주문 생성 시간)
df_selected['delivery_time_hours'] = (df_selected['Dropoff done at'] - df_selected['Created At']).dt.total_seconds() / 3600

# 배달 소요 시간 분포 확인
import seaborn as sns
import matplotlib.pyplot as plt

sns.histplot(df_selected['delivery_time_hours'].dropna(), bins=20)
plt.title('Delivery Time Distribution (Hours)')
plt.xlabel('Delivery Time (Hours)')
plt.ylabel('Frequency')
plt.show()


In [ ]:
from geopy.distance import geodesic  # geodesic 함수 임포트
import seaborn as sns
import pandas as pd

# 결측치가 없는 행만 선택
df_filtered = df_selected.dropna(subset=['Sender Latitude', 'Sender Longitude', 'Consignee Latitude', 'Consignee Longitude'])

# 거리 계산 (픽업 지점과 드롭오프 지점 간의 거리, km 단위)
df_filtered['distance_km'] = df_filtered.apply(
    lambda row: geodesic(
        (row['Sender Latitude'], row['Sender Longitude']),
        (row['Consignee Latitude'], row['Consignee Longitude'])
    ).km, axis=1)

# 거리 분포 확인
sns.histplot(df_filtered['distance_km'], bins=20)
plt.title('Distance Distribution (KM)')
plt.xlabel('Distance (KM)')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# 거리 데이터 통계 확인
print(df_filtered['distance_km'].describe())

# 박스 플롯으로 이상치 시각화
sns.boxplot(df_filtered['distance_km'])
plt.title('Distance Distribution with Outliers')
plt.xlabel('Distance (KM)')
plt.show()


In [ ]:
# 거리가 0km 또는 비현실적으로 큰 값을 제거 (예: 1km 이상 1000km 미만으로 제한)
df_filtered = df_filtered[(df_filtered['distance_km'] > 0) & (df_filtered['distance_km'] < 1000)]

# 이상치 제거 후 거리 분포 재시각화
sns.histplot(df_filtered['distance_km'], bins=20)
plt.title('Distance Distribution (KM) After Outlier Removal')
plt.xlabel('Distance (KM)')
plt.ylabel('Frequency')
plt.show()


In [ ]:
from sklearn.cluster import KMeans

# 클러스터링을 위한 데이터 선택 (무게와 거리를 사용)
X = df_filtered[['Weight', 'distance_km']].dropna()

# K-Means 클러스터링 모델 적용
kmeans = KMeans(n_clusters=3)
df_filtered['cluster'] = kmeans.fit_predict(X)

# 클러스터 시각화
sns.scatterplot(data=df_filtered, x='Weight', y='distance_km', hue='cluster', palette='viridis')
plt.title('Clustering of Cargo by Weight and Distance')
plt.xlabel('Weight')
plt.ylabel('Distance (KM)')
plt.show()


In [ ]:
# 클러스터별 통계 요약
cluster_stats = df_filtered.groupby('cluster').agg({
    'Weight': ['mean', 'median', 'std'],
    'distance_km': ['mean', 'median', 'std']
})
print(cluster_stats)


In [ ]:
# 클러스터별 무게 분포
sns.boxplot(data=df_filtered, x='cluster', y='Weight')
plt.title('Weight Distribution by Cluster')
plt.show()

# 클러스터별 거리 분포
sns.boxplot(data=df_filtered, x='cluster', y='distance_km')
plt.title('Distance Distribution by Cluster')
plt.show()


In [ ]:
# 클러스터 0 데이터만 필터링
df_cluster_0 = df_filtered[df_filtered['cluster'] == 0]

# 클러스터 0 데이터 확인
print(df_cluster_0.head())


In [ ]:
# 클러스터 0의 무게와 거리 통계
cluster_0_stats = df_cluster_0[['Weight', 'distance_km']].describe()
print(cluster_0_stats)

# 클러스터 0의 무게 분포 시각화
sns.histplot(df_cluster_0['Weight'], bins=20)
plt.title('Weight Distribution in Cluster 0')
plt.xlabel('Weight')
plt.ylabel('Frequency')
plt.show()

# 클러스터 0의 거리 분포 시각화
sns.histplot(df_cluster_0['distance_km'], bins=20)
plt.title('Distance Distribution in Cluster 0')
plt.xlabel('Distance (KM)')
plt.ylabel('Frequency')
plt.show()


In [ ]:
import numpy as np
from geopy.distance import geodesic

# 자카르타 내 임의의 라이더 좌표 생성 (위도 및 경도 범위 지정)
num_riders = len(df_cluster_0)  # 클러스터 0의 데이터 수에 맞춰 라이더 수 생성
np.random.seed(42)  # 재현 가능한 결과를 위해 시드 설정

# 자카르타 내 위도 및 경도 범위에서 임의 좌표 생성
rider_latitudes = np.random.uniform(low=-6.3670, high=-6.1256, size=num_riders)
rider_longitudes = np.random.uniform(low=106.7200, high=106.9900, size=num_riders)

# 라이더 위치를 데이터프레임에 추가
df_cluster_0['rider_latitude'] = rider_latitudes
df_cluster_0['rider_longitude'] = rider_longitudes

# 라이더와 픽업 지점 간의 거리 계산 (km 단위)
df_cluster_0['rider_to_pickup_distance'] = df_cluster_0.apply(
    lambda row: geodesic(
        (row['rider_latitude'], row['rider_longitude']),
        (row['Sender Latitude'], row['Sender Longitude'])
    ).km, axis=1)

# 라이더와 픽업 지점 간 거리 분포 시각화
sns.histplot(df_cluster_0['rider_to_pickup_distance'], bins=20)
plt.title('Distance from Rider to Pickup Location (KM)')
plt.xlabel('Distance (KM)')
plt.ylabel('Frequency')
plt.show()


In [ ]:
pip install google-maps-routeoptimization


In [ ]:
pip install google-cloud-storage


In [ ]:
import numpy as np

# Cluster 0에 해당하는 데이터프레임이 df_cluster_0이라고 가정
# Cluster 0에 라이더 좌표 추가

# 자카르타 내 임의의 라이더 좌표 생성 (위도 및 경도 범위 지정)
num_riders = len(df_cluster_0)  # Cluster 0의 데이터 수에 맞춰 라이더 수 생성
np.random.seed(42)  # 재현 가능한 결과를 위해 시드 설정

# 자카르타 내 위도 및 경도 범위에서 임의 좌표 생성
rider_latitudes = np.random.uniform(low=-6.3670, high=-6.1256, size=num_riders)
rider_longitudes = np.random.uniform(low=106.7200, high=106.9900, size=num_riders)

# 라이더 위치를 데이터프레임에 추가
df_cluster_0['rider_latitude'] = rider_latitudes
df_cluster_0['rider_longitude'] = rider_longitudes

# 좌표가 잘 추가되었는지 확인
print(df_cluster_0[['rider_latitude', 'rider_longitude']].head())


In [24]:
from google.cloud import storage

# 프로젝트 ID 수동으로 설정
client = storage.Client(project="341524124675")  # 'your-project-id'를 실제 프로젝트 ID로 대체

# 버킷 리스트 가져오기
buckets = list(client.list_buckets())

# 버킷 이름 출력
for bucket in buckets:
    print(bucket.name)


In [25]:
from google.maps import routeoptimization_v1 as ro
from google.type import latlng_pb2 as latlng  # LatLng를 가져오기

# 샘플 데이터를 추출하여 df_sample 생성 (예: 100개의 샘플 데이터 사용)
df_sample = df_cluster_0.sample(10, random_state=42)

In [ ]:
pip install google-maps-routeoptimization


In [ ]:
# 메서드 목록 출력
print(dir(client))


In [ ]:
pip install googlemaps


In [82]:
pip install python-dotenv


  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [83]:
import googlemaps
import pandas as pd

import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

# Google Maps API 키 가져오기
google_maps_api_key = os.getenv('GOOGLE_MAPS_API_KEY')

# 샘플 데이터에서 첫 번째 행을 사용 (추가 테스트를 위해 반복문 사용 가능)
sample_row = df_sample.iloc[0]

# 라이더 위치 (출발지)
origin = (sample_row['rider_latitude'], sample_row['rider_longitude'])

# 픽업 위치
pickup_location = (sample_row['Sender Latitude'], sample_row['Sender Longitude'])

# 배송 위치 (목적지)
delivery_location = (sample_row['Consignee Latitude'], sample_row['Consignee Longitude'])

# Directions API를 통해 라이더 -> 픽업 지점 -> 배송 목적지 경로 요청
directions_result = gmaps.directions(origin,
                                     delivery_location,
                                     waypoints=[pickup_location],  # 경유지: 픽업 지점
                                     optimize_waypoints=True,  # 경유지 최적화
                                     mode="driving")

# 결과 출력
print(directions_result)


[{'bounds': {'northeast': {'lat': -6.2203158, 'lng': 106.8969582}, 'southwest': {'lat': -6.388872699999999, 'lng': 106.7373652}}, 'copyrights': 'Map data ©2024 Google', 'legs': [{'distance': {'text': '8.2 km', 'value': 8223}, 'duration': {'text': '20 mins', 'value': 1212}, 'end_address': 'Jl. Dewi Sartika No.1, RT.5/RW.12, Kuningan, Cawang, Kec. Kramat jati, Kota Jakarta Timur, Daerah Khusus Ibukota Jakarta 13630, Indonesia', 'end_location': {'lat': -6.252152700000001, 'lng': 106.8646154}, 'start_address': 'Jl. H. R. Rasuna Said No.2, RT.2/RW.5, Karet Kuningan, Kecamatan Setiabudi, Kota Jakarta Selatan, Daerah Khusus Ibukota Jakarta 12940, Indonesia', 'start_location': {'lat': -6.2203158, 'lng': 106.8330094}, 'steps': [{'distance': {'text': '0.1 km', 'value': 104}, 'duration': {'text': '1 min', 'value': 42}, 'end_location': {'lat': -6.2208011, 'lng': 106.8324117}, 'html_instructions': 'Head <b>west</b>', 'polyline': {'points': '~{}d@iy`kSDb@@FRf@DJ@J@DDFHDB@F?JCJADC'}, 'start_location'

In [89]:
import os
import googlemaps
import pandas as pd
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

# .env 파일에 저장된 Google Maps API 키를 불러옴
google_maps_api_key = os.getenv('GOOGLE_MAPS_API_KEY')

# Google Maps API 클라이언트 생성
gmaps = googlemaps.Client(key=google_maps_api_key)

# df_sample에서 데이터 추출하여 루프를 통해 라이더별로 거리 및 소요 시간 계산
route_distances = []
route_durations = []

for index, row in df_sample.iterrows():
    try:
        # 라이더 위치 (출발지)
        origin = (row['rider_latitude'], row['rider_longitude'])
        
        # 픽업 위치
        pickup_location = (row['Sender Latitude'], row['Sender Longitude'])
        
        # 배송 위치 (목적지)
        delivery_location = (row['Consignee Latitude'], row['Consignee Longitude'])
        
        # Directions API를 통해 경로 계산
        directions_result = gmaps.directions(origin,
                                             delivery_location,
                                             waypoints=[pickup_location],  # 경유지: 픽업 지점
                                             optimize_waypoints=True,
                                             mode="driving")
        
        # Directions API 결과가 비어 있는지 확인
        if not directions_result or 'legs' not in directions_result[0]:
            print(f"No route found for row {index}")
            route_distances.append(None)
            route_durations.append(None)
            continue
        
        # Directions API 결과에서 경로 거리와 소요 시간 추출
        route_distance = directions_result[0]['legs'][0]['distance']['value']  # 총 거리 (미터 단위)
        route_duration = directions_result[0]['legs'][0]['duration']['value']  # 총 소요 시간 (초 단위)
        
        # 리스트에 결과 저장
        route_distances.append(route_distance / 1000)  # 킬로미터 단위로 변환
        route_durations.append(route_duration / 60)  # 분 단위로 변환
    
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        route_distances.append(None)
        route_durations.append(None)

# df_cluster_0에 거리 및 소요 시간 추가
df_cluster_0['route_distance'] = route_distances
df_cluster_0['route_duration'] = route_durations

# 추가: 경로가 없는 행을 삭제하거나 특정 값으로 대체할 수 있음
df_cluster_0.dropna(subset=['route_distance', 'route_duration'], inplace=True)


ValueError: Must provide API key or enterprise credentials when creating client.

In [ ]:
route_distances = []
route_durations = []

# 전체 df_cluster_0 데이터프레임에 대해 경로 계산 적용
for index, row in df_cluster_0.iterrows():
    try:
        # 라이더 위치 (출발지)
        origin = (row['rider_latitude'], row['rider_longitude'])
        
        # 픽업 위치
        pickup_location = (row['Sender Latitude'], row['Sender Longitude'])
        
        # 배송 위치 (목적지)
        delivery_location = (row['Consignee Latitude'], row['Consignee Longitude'])
        
        # Directions API를 통해 경로 계산
        directions_result = gmaps.directions(origin,
                                             delivery_location,
                                             waypoints=[pickup_location],  # 경유지: 픽업 지점
                                             optimize_waypoints=True,
                                             mode="driving")
        
        # Directions API 결과가 비어 있는지 확인
        if not directions_result or 'legs' not in directions_result[0]:
            print(f"No route found for row {index}")
            route_distances.append(None)
            route_durations.append(None)
            continue
        
        # Directions API 결과에서 경로 거리와 소요 시간 추출
        route_distance = directions_result[0]['legs'][0]['distance']['value']  # 총 거리 (미터 단위)
        route_duration = directions_result[0]['legs'][0]['duration']['value']  # 총 소요 시간 (초 단위)
        
        # 리스트에 결과 저장
        route_distances.append(route_distance / 1000)  # 킬로미터 단위로 변환
        route_durations.append(route_duration / 60)  # 분 단위로 변환
    
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        route_distances.append(None)
        route_durations.append(None)

# 전체 df_cluster_0에 거리 및 소요 시간 추가
df_cluster_0['route_distance'] = route_distances
df_cluster_0['route_duration'] = route_durations


In [ ]:
df_cluster_0['route_distance']

In [ ]:
df_cluster_0['route_duration']

In [ ]:
df_cluster_0.dropna(subset=['route_distance', 'route_duration'], inplace=True)
# 또는
df_cluster_0.fillna({'route_distance': df_cluster_0['route_distance'].mean(),
                    'route_duration': df_cluster_0['route_duration'].mean()}, inplace=True)


In [66]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 필요한 feature와 target 설정
features = df_cluster_0[['distance_to_pickup', 'distance_to_delivery', 'route_distance', 'route_duration']]
target = df_cluster_0['assigned']  # 배차 여부를 목표 변수로 설정

# 데이터를 학습용과 테스트용으로 나누기
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

# 스케일링 (딥러닝 모델에 유용)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 랜덤 포레스트 모델 구축
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# 테스트 데이터로 예측 수행
rf_predictions = rf_model.predict(X_test)

# 성능 평가
rf_accuracy = accuracy_score(y_test, rf_predictions)
print(f"랜덤 포레스트 모델 정확도: {rf_accuracy}")
print("분류 리포트:\n", classification_report(y_test, rf_predictions))


In [ ]:
import xgboost as xgb

# XGBoost 모델 구축
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)

# 테스트 데이터로 예측 수행
xgb_predictions = xgb_model.predict(X_test)

# 성능 평가
xgb_accuracy = accuracy_score(y_test, xgb_predictions)
print(f"XGBoost 모델 정확도: {xgb_accuracy}")
print("분류 리포트:\n", classification_report(y_test, xgb_predictions))


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 딥러닝 모델 구축
dl_model = Sequential()

# 입력층
dl_model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))

# 은닉층
dl_model.add(Dense(32, activation='relu'))

# 출력층 (이진 분류이므로 sigmoid 사용)
dl_model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
dl_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
dl_model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_split=0.2)

# 테스트 데이터로 예측 수행
dl_predictions = dl_model.predict(X_test_scaled)
dl_predictions = (dl_predictions > 0.5).astype(int)  # 0.5 기준으로 이진 분류

# 성능 평가
dl_accuracy = accuracy_score(y_test, dl_predictions)
print(f"딥러닝 모델 정확도: {dl_accuracy}")
print("분류 리포트:\n", classification_report(y_test, dl_predictions))


In [ ]:
import matplotlib.pyplot as plt
from xgboost import plot_importance

# Feature importance 출력
plot_importance(xgb_model)
plt.show()


In [ ]:
pip install imblearn

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# XGBoost 모델 다시 학습
xgb_model_res = xgb.XGBClassifier(random_state=42)
xgb_model_res.fit(X_train_res, y_train_res)

# 예측 및 성능 평가
xgb_predictions_res = xgb_model_res.predict(X_test)
xgb_accuracy_res = accuracy_score(y_test, xgb_predictions_res)
print(f"XGBoost 모델 (SMOTE 적용) 정확도: {xgb_accuracy_res}")


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.3]
}

grid_search = GridSearchCV(estimator=xgb.XGBClassifier(random_state=42),
                           param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print(f"최적 파라미터: {grid_search.best_params_}")
best_xgb_model = grid_search.best_estimator_

# 최적 모델로 예측 수행 및 평가
xgb_predictions_gs = best_xgb_model.predict(X_test)
xgb_accuracy_gs = accuracy_score(y_test, xgb_predictions_gs)
print(f"최적 XGBoost 모델 정확도: {xgb_accuracy_gs}")


In [ ]:
from sklearn.model_selection import cross_val_score

# 교차 검증
cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='accuracy')
print(f"5-겹 교차 검증 정확도: {cv_scores.mean()}")


In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# 신경망 모델 구축
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # 이진 분류의 경우 sigmoid 사용

# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# 모델 평가
_, accuracy = model.evaluate(X_test, y_test)
print(f"신경망 모델 정확도: {accuracy}")


In [ ]:
from sklearn.feature_selection import RFE

# XGBoost 모델 기반 RFE
selector = RFE(xgb_model, n_features_to_select=10)
selector = selector.fit(X_train, y_train)

# 선택된 특징으로 다시 모델 학습
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

xgb_model.fit(X_train_selected, y_train)
predictions = xgb_model.predict(X_test_selected)
accuracy = accuracy_score(y_test, predictions)
print(f"특징 선택 후 XGBoost 모델 정확도: {accuracy}")
